Tessaract

In [1]:
! apt install tesseract-ocr
! apt install libtesseract-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,720 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 145483 files and directories currently installed.)
Preparing to unpack .../tesserac

#Pytessaract

In [2]:
! pip install Pillow
! pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13945 sha256=d637f0ef29f81a252b39541730cd56e7aafa39c83ee2f4bd5d32995a7e26a82a
  Stored in directory: /root/.cache/pip/wheels/81/20/7e/1dd0daad1575d5260916bb1e9781246430647adaef4b3ca3b3
Successfully built pytesseract


In [3]:
import pytesseract
from PIL import ImageEnhance, ImageFilter, Image

In [4]:

def get_acc(directory, path):
#def get_acc(/content/out, /content/out/001.jpg ):
    font     = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5
    fontColor  = (255,0,0)
    lineType = 1
    path = directory+path
    #path = "/content/out/001.jpg"
    #path = '/content/000.jpg'
    #path = args['/content/out/001.jpg']
    #op_path = args['/content/sample_data']

    op_path = directory
    if op_path[-1]!='/':
    	op_path.append('/')


    #Threshold
    image = cv2.imread(path)

    height,width,channel = image.shape

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    T = threshold_local(gray, 15, offset = 6, method = "gaussian") # generic, mean, median, gaussian
    thresh = (gray > T).astype("uint8") * 255
    thresh = ~thresh


    #Dilation
    kernel =np.ones((1,1), np.uint8)
    ero = cv2.erode(thresh, kernel, iterations= 1)
    img_dilation = cv2.dilate(ero, kernel, iterations=1)

    # Remove noise
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(img_dilation, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    final = np.zeros((labels.shape), np.uint8)
    for i in range(0, nlabels - 1):
        if sizes[i] >= 10:   #filter small dotted regions
            final[labels == i + 1] = 255

    kern = np.ones((5,15), np.uint8)
    img_dilation = cv2.dilate(final, kern, iterations = 1)
    contours, hierarchy = cv2.findContours(img_dilation, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    rects = map(lambda c: cv2.boundingRect(c), contours)
    sorted_rects = sorted(rects, key =lambda r: r[0])
    sorted_rects = sorted(sorted_rects, key =lambda r: r[1])

    tt = image.copy()
    dictionary = {}
    etfo=''
    for i,rect in enumerate(sorted_rects):
        temp_dic = {}
        x,y,w,h = rect
        if(w<20 or h<20):
            continue
        temp_dic['coords'] = [x,y,w,h]
        words = []
        temp = tt[y:y+h, x:x+w]
        temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
        hi = pytesseract.image_to_data(temp, config=r'--psm 6')
        hi = hi.split()
        ind = 22
        while(True):
            if (ind>len(hi)):
                break
            if(int(hi[ind])==-1):
                ind+=11
            else:

                tem = {}
                tem['confidence'] = hi[ind]
                tem['text'] = hi[ind+1]
                etfo=etfo+hi[ind+1]
                etfo=etfo+" "
                x+=len(hi[ind+1])*20
                ind+=12
                words.append(tem)
        temp_dic['words'] = words
        etfo=etfo+'\n'
        #cvw.rectangle(image, rect, cvw.Color.GREEN, thickness=1)
        dictionary[i] = temp_dic


    cv2.imwrite(op_path+"result.jpg", image)
    return json.dumps(dictionary),etfo

    #cv2_imshow("res",image)


In [5]:
import cv2
import argparse
import numpy as np
!pip install opencv-wrapper
import opencv_wrapper as cvw
from skimage.filters import threshold_local
import json

     |████████████████████████████████| 17.3MB 252kB/s 
     |████████████████████████████████| 25.4MB 1.6MB/s 
  Created wheel for opencv-wrapper: filename=opencv_wrapper-0.2.3-py2.py3-none-any.whl size=18580 sha256=7efee24fa397be04b3c0665afe6ef2c6b9f6038ce2d90bad5c2eb60bef675d6c
  Stored in directory: /root/.cache/pip/wheels/74/fc/22/03106b3bcb2516e69317562a2aa4ceb94b0d5813a448edd928
Successfully built opencv-wrapper
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement numpy~=1.19.2, but you'll have numpy 1.16.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4
  Found existing i

In [6]:
#Provide path of test image present in the created folder at “gson_data,etfo”

gson_data, etfo = get_acc('/content/out/','000.jpg')
print(gson_data)


{"1": {"coords": [68, 30, 258, 35], "words": [{"confidence": "93", "text": "tan"}, {"confidence": "89", "text": "woon"}, {"confidence": "91", "text": "yann"}]}, "3": {"coords": [66, 93, 361, 22], "words": [{"confidence": "93", "text": "BOOK"}, {"confidence": "83", "text": "TA_K"}, {"confidence": "90", "text": "(TAMAN"}, {"confidence": "91", "text": "DAYA)"}, {"confidence": "90", "text": "SDN"}, {"confidence": "47", "text": "BHD"}]}, "5": {"coords": [158, 142, 231, 24], "words": [{"confidence": "7", "text": "$5.57"}, {"confidence": "92", "text": "&"}, {"confidence": "83", "text": "58,"}, {"confidence": "92", "text": "JALAN"}, {"confidence": "88", "text": "SAGU"}, {"confidence": "80", "text": "18,"}]}, "8": {"coords": [185, 167, 121, 23], "words": [{"confidence": "88", "text": "TAMAN"}, {"confidence": "63", "text": "DAYA"}]}, "10": {"coords": [157, 191, 184, 22], "words": [{"confidence": "86", "text": "81100"}, {"confidence": "92", "text": "JOHOR"}, {"confidence": "35", "text": "BAHRU."}

In [7]:
import argparse
import torch
import json
from torch import nn

In [8]:
class MyModel0(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=2, bidirectional=True)
        self.linear = nn.Linear(hidden_size * 2, 5)

    def forward(self, inpt):
        embedded = self.embed(inpt)
        feature, _ = self.lstm(embedded)
        oupt = self.linear(feature)
        return oupt

In [9]:
#Installing Colorama.
!pip install colorama

In [10]:
import json
import os
import random
from os import path
from string import ascii_uppercase, digits, punctuation

import colorama
import numpy
import regex

In [11]:

VOCAB= ascii_uppercase+digits+punctuation+" \t\n"

In [12]:
print(etfo)

tan woon yann 
BOOK TA_K (TAMAN DAYA) SDN BHD 
$5.57 & 58, JALAN SAGU 18, 
TAMAN DAYA 
81100 JOHOR BAHRU. 
db STEHT 
25/12/2018 8:13:39 PM 
CASH BILL 
CODE/DESC 
Qry 
Rour ding Adjustment 
Round: :d Total (RM); 
Cash 9 ' 40.00 CHANGE 00: Do 0 GOODS SOLD ARE NOT RETURINAT aK 
"Wap, 00, 
?-00 
HOPE, f, ROB ate ls oo fa 7 0, GOae yet. ata! 



In [14]:
#Creating text_tensor using pytorch

device= 'cpu'
hidden_size = 256

device= torch.device('cpu')
def get_test_data():
    
    text = etfo
    text_tensor = torch.zeros(len(text), 1, dtype=torch.long)
    text_tensor[:, 0] = torch.LongTensor([VOCAB.find(c) for c in text])

    return text_tensor.to(device)

text_tensor = get_test_data()

In [15]:

print(text_tensor.shape)

torch.Size([340, 1])


In [16]:
def pred_to_dict(text, pred, prob):
    res = {"company": ("", 0), "date": ("", 0), "address": ("", 0), "total": ("", 0)}
    keys = list(res.keys())

    seps = [0] + (numpy.nonzero(numpy.diff(pred))[0] + 1).tolist() + [len(pred)]
    for i in range(len(seps) - 1):
        pred_class = pred[seps[i]] - 1
        if pred_class == -1:
            continue

        new_key = keys[pred_class]
        new_prob = prob[seps[i] : seps[i + 1]].max()
        if new_prob > res[new_key][1]:
            res[new_key] = (text[seps[i] : seps[i + 1]], new_prob)

    return {k: regex.sub(r"[\t\n]", " ", v[0].strip()) for k, v in res.items()}


In [20]:
#Check for the less error-free size of "torch.size" 
#for this time i had to check for 5 times

print(text_tensor.shape)
for i in range(len(text_tensor)-1):
  if text_tensor[i]<0 or text_tensor[i]>70:
    text_tensor = torch.cat([text_tensor[0:i], text_tensor[i+1:]])

torch.Size([276, 1])


#Testing

In [22]:
#Making use of trained model “model.pth” from Zhang for testing on images.

def test():
    
    model = MyModel0(len(VOCAB), 16, hidden_size).to(device)
  

    model.load_state_dict(torch.load("/content/out/model.pth"))

    model.eval()
  
    with torch.no_grad():
            oupt = model(text_tensor)
            prob = torch.nn.functional.softmax(oupt, dim=2)
            prob, pred = torch.max(prob, dim=2)
            prob = prob.squeeze().cpu().numpy()
            pred = pred.squeeze().cpu().numpy()
            real_text = etfo
            result = pred_to_dict(real_text, pred, prob)

            with open("/content/out/result" + 'result' + ".json", "w", encoding="utf-8") as json_opened:
                json.dump(result, json_opened, indent=4)

            print(result)
            #print(key)


if __name__ == "__main__":
    test()

{'company': 'woon yann  BOOK TA_K (TAMAN DAY', 'date': '.  db STEH', 'address': 'DN BHD  $5.57 & 58, JALAN SAGU 18,  TAMAN DAYA  81100 JOHOR BAHR', 'total': ''}
